In [103]:
import jax
import jax.numpy as jnp

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import dataframe_image as dfi

In [104]:
def convert(df):
    def _convert(string):
        l, s, o = string.split(".")
        l = "l" if l == "1" else ""
        s = "s" if s == "1" else ""
        o = "o" if o == "1" else ""
        ans = l+s+o
        if ans == "":
            return "stat"
        else:
            return ans
    df = df.copy()
    df.columns = map(_convert, df.columns)
    df.index = map(_convert, df.index)
    return df

In [105]:
latent_key = "203" # 2000, 200, 203, 20000
data_key = "303"  # 3000, 300, 303, 30000
rmse_inducing = convert(pd.read_csv(f"/home/patel_zeel/GP_Neurips/inducing_rmse_{latent_key}_{data_key}.csv", index_col=0, header=0))
# rmse_lgp = convert(pd.read_csv(f"/home/patel_zeel/GP_Neurips/lgp_inducing_rmse_{latent_key}_{data_key}.csv", index_col=0, header=0))
rmse = convert(pd.read_csv(f"/home/patel_zeel/GP_Neurips/rmse_{latent_key}_{data_key}.csv", index_col=0, header=0))
nlpd = convert(pd.read_csv(f"/home/patel_zeel/GP_Neurips/nlpd_{latent_key}_{data_key}.csv", index_col=0, header=0))
nlpd_inducing = convert(pd.read_csv(f"/home/patel_zeel/GP_Neurips/inducing_nlpd_{latent_key}_{data_key}.csv", index_col=0, header=0))
# nlpd_lgp = convert(pd.read_csv(f"/home/patel_zeel/GP_Neurips/lgp_inducing_nlpd_[  0 {latent_key}]_[  0 {data_key}].csv", index_col=0, header=0))

In [106]:
# highlight lowest vales in each column
rmse_style = rmse.style.highlight_min(axis=0, color='green')
rmse_style_inducing = rmse_inducing.style.highlight_min(axis=0, color='green')
# rmse_style_lgp = rmse_lgp.style.highlight_min(axis=0, color='green')
names = ["rmse_style", "rmse_style_inducing", "rmse_style_lgp"]
styles = [rmse_style, rmse_style_inducing]
# for name, style in zip(names, styles):
#         dfi.export(style, f"{name}_{latent_key}_{data_key}.png")
display(rmse_style, 
        rmse_style_inducing, 
        # rmse_style_lgp
        )

,lso,ls,lo,l,so,s,o,stat
lso,0.151192,0.316537,0.149733,0.329418,0.147985,0.333393,0.142873,0.318083
ls,0.169581,0.323288,0.168993,0.336628,0.166555,0.333449,0.163402,0.318550
lo,0.148888,0.314697,0.144761,0.326927,0.143647,0.331741,0.141822,0.316864
l,0.170012,0.314649,0.168218,0.327900,0.180261,0.330965,0.176951,0.315272
so,0.150701,0.315894,0.150185,0.328770,0.147673,0.331416,0.142480,0.316947
s,0.198898,0.316190,0.185188,0.329003,0.148733,0.331491,0.142449,0.317188
o,0.148688,0.313308,0.147121,0.326323,0.142975,0.329506,0.131176,0.312225
stat,0.155096,0.314517,0.153404,0.327354,0.151052,0.330469,0.141098,0.311421


,lso,ls,lo,l,so,s,o,stat
lso,0.150000,0.315169,0.150348,0.331611,0.147396,0.333845,0.142774,0.316458
ls,0.156060,0.313207,0.168173,0.327590,0.150684,0.330896,0.140363,0.315860
lo,0.149346,0.314493,0.148055,0.326578,0.146620,0.326272,0.143074,0.314834
l,0.162074,0.313955,0.182870,0.327904,0.149497,0.328343,0.140998,0.311655
so,0.152972,0.329198,0.148476,0.345378,0.148337,0.333662,0.144056,0.312042
s,0.155027,0.314669,0.154194,0.328074,0.150491,0.328964,0.140449,0.311462
o,0.148956,0.313017,0.149379,0.327252,0.144351,0.328715,0.134421,0.310947
stat,0.155096,0.314518,0.153381,0.327354,0.151053,0.330469,0.141098,0.311421


In [107]:
nlpd_style = nlpd.style.highlight_min(axis=0, color='green')
nlpd_style_inducing = nlpd_inducing.style.highlight_min(axis=0, color='green')
# nlpd_style_lgp = nlpd_lgp.style.highlight_min(axis=0, color='green')
display(nlpd_style, 
        nlpd_style_inducing, 
        # nlpd_style_lgp
        )

,lso,ls,lo,l,so,s,o,stat
lso,-1.009844,0.295882,-0.973996,0.341300,-1.030425,0.350517,-1.068875,0.306638
ls,-0.658349,0.307676,-0.669248,0.351933,-0.695458,0.347633,-0.734332,0.304075
lo,-1.008376,0.292879,-1.005495,0.339370,-1.039528,0.345874,-1.072317,0.302222
l,-0.661688,0.291984,-0.671966,0.336430,-0.489947,0.343897,-0.502441,0.296978
so,-1.004781,0.293691,-0.957249,0.339282,-1.028073,0.345374,-1.068591,0.302307
s,-0.319579,0.292514,-0.331499,0.336640,-0.460201,0.342942,-0.496301,0.299753
o,-1.004658,0.290500,-0.997695,0.336309,-1.034087,0.345283,-1.111901,0.288990
stat,-0.437045,0.290705,-0.446788,0.332961,-0.453262,0.343397,-0.514114,0.286275


,lso,ls,lo,l,so,s,o,stat
lso,-0.972619,0.289789,-0.920859,0.393834,-0.974689,0.471058,-1.059575,0.303092
ls,-0.433027,0.286502,-0.506382,0.335024,-0.457539,0.345308,-0.517526,0.295646
lo,-0.981965,0.289227,-0.952991,0.355208,-0.970660,0.358976,-1.031759,0.309355
l,-0.594620,0.289337,-0.550284,0.335125,-0.466658,0.327715,-0.512730,0.286885
so,-0.947893,0.357321,-0.977154,0.390700,-0.974783,0.349845,-1.036200,0.289200
s,-0.433171,0.291045,-0.443336,0.336055,-0.460409,0.340058,-0.519727,0.286056
o,-0.984644,0.308664,-0.943503,0.325120,-0.977577,0.375086,-1.072140,0.293493
stat,-0.437047,0.290706,-0.446948,0.332961,-0.453263,0.343397,-0.514115,0.286275


In [108]:
# merge rmse and nlpd by creating an extra column header
def merge(df1, df2):
    df1 = df1.copy()
    df2 = df2.copy()
    df1.columns = df1.columns + "_rmse"
    df2.columns = df2.columns + "_nlpd"
    df = pd.concat([df1, df2], axis=1)
    # sort columns
    new_cols = []
    for a,b in zip(df1.columns, df2.columns):
        new_cols.append(a)
        new_cols.append(b)
    df = df.reindex(new_cols, axis=1)
    return df

rmse_nlpd = merge(rmse, nlpd).style.highlight_min(axis=0, color='green')
rmse_nlpd

,lso_rmse,lso_nlpd,ls_rmse,ls_nlpd,lo_rmse,lo_nlpd,l_rmse,l_nlpd,so_rmse,so_nlpd,s_rmse,s_nlpd,o_rmse,o_nlpd,stat_rmse,stat_nlpd
lso,0.151192,-1.009844,0.316537,0.295882,0.149733,-0.973996,0.329418,0.341300,0.147985,-1.030425,0.333393,0.350517,0.142873,-1.068875,0.318083,0.306638
ls,0.169581,-0.658349,0.323288,0.307676,0.168993,-0.669248,0.336628,0.351933,0.166555,-0.695458,0.333449,0.347633,0.163402,-0.734332,0.318550,0.304075
lo,0.148888,-1.008376,0.314697,0.292879,0.144761,-1.005495,0.326927,0.339370,0.143647,-1.039528,0.331741,0.345874,0.141822,-1.072317,0.316864,0.302222
l,0.170012,-0.661688,0.314649,0.291984,0.168218,-0.671966,0.327900,0.336430,0.180261,-0.489947,0.330965,0.343897,0.176951,-0.502441,0.315272,0.296978
so,0.150701,-1.004781,0.315894,0.293691,0.150185,-0.957249,0.328770,0.339282,0.147673,-1.028073,0.331416,0.345374,0.142480,-1.068591,0.316947,0.302307
s,0.198898,-0.319579,0.316190,0.292514,0.185188,-0.331499,0.329003,0.336640,0.148733,-0.460201,0.331491,0.342942,0.142449,-0.496301,0.317188,0.299753
o,0.148688,-1.004658,0.313308,0.290500,0.147121,-0.997695,0.326323,0.336309,0.142975,-1.034087,0.329506,0.345283,0.131176,-1.111901,0.312225,0.288990
stat,0.155096,-0.437045,0.314517,0.290705,0.153404,-0.446788,0.327354,0.332961,0.151052,-0.453262,0.330469,0.343397,0.141098,-0.514114,0.311421,0.286275


In [109]:
rmse_nlpd_inducing = merge(rmse_inducing, nlpd_inducing).style.highlight_min(axis=0, color='green')
rmse_nlpd_inducing

,lso_rmse,lso_nlpd,ls_rmse,ls_nlpd,lo_rmse,lo_nlpd,l_rmse,l_nlpd,so_rmse,so_nlpd,s_rmse,s_nlpd,o_rmse,o_nlpd,stat_rmse,stat_nlpd
lso,0.150000,-0.972619,0.315169,0.289789,0.150348,-0.920859,0.331611,0.393834,0.147396,-0.974689,0.333845,0.471058,0.142774,-1.059575,0.316458,0.303092
ls,0.156060,-0.433027,0.313207,0.286502,0.168173,-0.506382,0.327590,0.335024,0.150684,-0.457539,0.330896,0.345308,0.140363,-0.517526,0.315860,0.295646
lo,0.149346,-0.981965,0.314493,0.289227,0.148055,-0.952991,0.326578,0.355208,0.146620,-0.970660,0.326272,0.358976,0.143074,-1.031759,0.314834,0.309355
l,0.162074,-0.594620,0.313955,0.289337,0.182870,-0.550284,0.327904,0.335125,0.149497,-0.466658,0.328343,0.327715,0.140998,-0.512730,0.311655,0.286885
so,0.152972,-0.947893,0.329198,0.357321,0.148476,-0.977154,0.345378,0.390700,0.148337,-0.974783,0.333662,0.349845,0.144056,-1.036200,0.312042,0.289200
s,0.155027,-0.433171,0.314669,0.291045,0.154194,-0.443336,0.328074,0.336055,0.150491,-0.460409,0.328964,0.340058,0.140449,-0.519727,0.311462,0.286056
o,0.148956,-0.984644,0.313017,0.308664,0.149379,-0.943503,0.327252,0.325120,0.144351,-0.977577,0.328715,0.375086,0.134421,-1.072140,0.310947,0.293493
stat,0.155096,-0.437047,0.314518,0.290706,0.153381,-0.446948,0.327354,0.332961,0.151053,-0.453263,0.330469,0.343397,0.141098,-0.514115,0.311421,0.286275


In [116]:
diff = (rmse_nlpd_inducing.data - rmse_nlpd.data)<0
print(diff.sum().sum()/diff.size)

# highlight True values
diff.style.applymap(lambda x: 'background-color: green' if x==True else '')

0.5234375


,lso_rmse,lso_nlpd,ls_rmse,ls_nlpd,lo_rmse,lo_nlpd,l_rmse,l_nlpd,so_rmse,so_nlpd,s_rmse,s_nlpd,o_rmse,o_nlpd,stat_rmse,stat_nlpd
lso,True,False,True,True,False,False,False,False,True,False,False,False,True,False,True,True
ls,True,False,True,True,True,False,True,True,True,False,True,True,True,False,True,True
lo,False,False,True,True,False,False,True,False,False,False,True,False,False,False,True,False
l,True,False,True,True,False,False,False,True,True,False,True,True,True,True,True,True
so,False,False,False,False,True,True,False,False,False,False,False,False,False,False,True,True
s,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True
o,False,False,True,False,False,False,False,True,False,False,True,False,False,False,True,False
stat,True,True,False,False,True,True,False,True,False,True,False,False,False,True,True,True
